# Import libraries

In [ ]:
import geotrellis.proj4._

import geotrellis.spark._
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.hadoop.formats._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.tiling.FloatingLayoutScheme

import geotrellis.vector._

import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.reader.GeoTiffReader
import geotrellis.raster.io.geotiff.tags.TiffTags
import geotrellis.raster.io.geotiff.GeoTiff

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.Path

import geotrellis.proj4._
import geotrellis.spark._
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.hadoop.formats._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.tiling.FloatingLayoutScheme
import geotrellis.vector._
import geotrellis.raster._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.reader.GeoTiffReader
import geotrellis.raster.io.geotiff.tags.TiffTags
import geotrellis.raster.io.geotiff.GeoTiff
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.Path


# Important variables (to run everytime we use Geotrellis)

In [ ]:
// implicit variable (important variables to run the functions in the geotrellis library)
implicit val sparkContext = sc

val rr = implicitly[RasterReader[HadoopGeoTiffRDD.Options, (ProjectedExtent, Tile)]]

sparkContext: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4b678ebd
rr: geotrellis.spark.io.RasterReader[geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options,(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = geotrellis.spark.io.RasterReader$$anon$1@6632c7a9


# Parameters

In [ ]:
val HdfsUrl = "hdfs://hupi-factory-02-01-01-01/"
val dataRepo = "user/factory02/thailand_workshop/data_usgs/"
val saveRepo = "user/factory02/thailand_workshop/ndvi/"

// Landsat8 
val landsatName = "LC08_L1TP_125052_20171231_20180103_01_T1"
val redBand = "B4"
val nirBand = "B5"


/*
// Landsat5 
val landsatName = "LT05_L1GS_125052_20070915_20161112_01_T2"
val redBand = "B3"
val nirBand = "B4"
*/

HdfsUrl: String = hdfs://hupi-factory-02-01-01-01/
dataRepo: String = user/factory02/thailand_workshop/data_usgs/
saveRepo: String = user/factory02/thailand_workshop/ndvi/
landsatName: String = LC08_L1TP_125052_20171231_20180103_01_T1
redBand: String = B4
nirBand: String = B5


In [ ]:
val saveName = HdfsUrl + saveRepo + landsatName

saveName: String = hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/ndvi/LC08_L1TP_125052_20171231_20180103_01_T1


# Delete the result if it's there already..

In [ ]:
val conf = sc.hadoopConfiguration  
val fs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI(HdfsUrl), conf)

conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
fs: org.apache.hadoop.fs.FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-1501002332_112, ugi=root (auth:SIMPLE)]]


In [ ]:
// Remove the image png of NDVI if it's there already
fs.delete(new Path(saveName + ".png"),true)

res7: Boolean = true


true

In [ ]:
// Remove the image tif of NDVI if it's there already
fs.delete(new Path(saveName + ".tif"),true)

res9: Boolean = true


true

# Compute NDVI

#### Firstly, we load GeoTiff from HDFS

In [ ]:
// To make the notebook run more efficiently 
val options =
HadoopGeoTiffRDD.Options(
  numPartitions = Some(100)
)

In [ ]:
// For Landsat 8 OLI, red band is 4th band and NIR band is 5th band 
val RedBand = HadoopGeoTiffRDD[ProjectedExtent, Tile](
      new Path(HdfsUrl + dataRepo + landsatName + "/" + landsatName + "_" + redBand + ".TIF"), 
      options).map(l => (l._1, l._2.convert(DoubleConstantNoDataCellType)))

val NIRBand = HadoopGeoTiffRDD[ProjectedExtent, Tile](
      new Path(HdfsUrl + dataRepo + landsatName + "/" + landsatName + "_" + nirBand + ".TIF"), 
      options).map(l => (l._1, l._2.convert(DoubleConstantNoDataCellType)))

RedBand: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[4] at map at <console>:111
NIRBand: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[9] at map at <console>:115


#### Then, we convert ProjectedExtent to SpatialKey...

In [ ]:
val (_, metadata_red) = RedBand.collectMetadata[SpatialKey](FloatingLayoutScheme())
val tiles_red = RedBand.tileToLayout[SpatialKey](metadata_red)

val (_, metadata_nir) = NIRBand.collectMetadata[SpatialKey](FloatingLayoutScheme())
val tiles_nir = NIRBand.tileToLayout[SpatialKey](metadata_nir)

metadata_red: geotrellis.spark.TileLayerMetadata[geotrellis.spark.SpatialKey] = TileLayerMetadata(float64,GridExtent(Extent(528585.0, 1156335.0, 758985.0, 1394415.0),30.0,30.0),Extent(528585.0, 1162785.0, 755415.0, 1394415.0),EPSG:32648,KeyBounds(SpatialKey(0,0),SpatialKey(29,30)))
tiles_red: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)] = ShuffledRDD[12] at reduceByKey at TileRDDMerge.scala:51
metadata_nir: geotrellis.spark.TileLayerMetadata[geotrellis.spark.SpatialKey] = TileLayerMetadata(float64,GridExtent(Extent(528585.0, 1156335.0, 758985.0, 1394415.0),30.0,30.0),Extent(528585.0, 1162785.0, 755415.0, 1394415.0),EPSG:32648,KeyBounds(SpatialKey(0,0),SpatialKey(29,30)))
tiles_nir: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raste...

#### ...and compute NDVI 

In [ ]:
val ndvi = (tiles_nir - tiles_red) / (tiles_nir + tiles_red).cache()

ndvi: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)] = MapPartitionsRDD[27] at mapValues at CombineMethods.scala:32


# Create colorMap

Values of NDVI are between -1 and 1. Here, the rule is that 

- if we have NDVI < 0 => ffffe5ff 
- if NDVI >=0 and NDVI < 0.1 => f7fcb9ff, etc.

We can change color by choosing in https://www.w3schools.com/colors/colors_picker.asp and add "ff" in the end 

In [ ]:
val ndviColormap = "0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff"

// Get color map from the application.conf settings file.
val colorMap = ColorMap.fromStringDouble(ndviColormap).get

ndviColormap: String = 0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff
colorMap: geotrellis.raster.render.ColorMap = geotrellis.raster.render.DoubleColorMap@793bffb3


# Save to HDFS 

In [ ]:
// To be able to save to HDFS, we always need to convert RDD[ProjectedExtent, Tile] to Raster[Tile]
// Here, we have many Tiles in this RDD, so we need to stitch all of them together
val raster_ndvi = ContextRDD(ndvi, metadata_red).stitch

raster_ndvi: geotrellis.raster.Raster[geotrellis.raster.Tile] = Raster(DoubleConstantNoDataArrayTile([D@4fb3c436,7680,7936),Extent(528585.0, 1156335.0, 758985.0, 1394415.0))


### Format tif 

In [ ]:
// Then we create GeoTiff[Raster, CRS] and write to HDFS
GeoTiff(raster_ndvi, metadata_red.crs).write(new Path(saveName + ".tif"))

No codec found for hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/ndvi/LC08_L1TP_125052_20171231_20180103_01_T1.tif, writing without compression.


### Format png 

In [ ]:
// To have a PNG, we need to use a Tile and render to PNG with a color map 
val ndvi_png = raster_ndvi.tile.renderPng(colorMap)

ndvi_png: geotrellis.raster.render.Png = Png([B@12b3d8cf)


In [ ]:
// Next, we write to HDFS
ndvi_png.write(new Path(saveName + ".png"))

No codec found for hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/ndvi/LC08_L1TP_125052_20171231_20180103_01_T1.png, writing without compression.


# We can also save PNG to server

Here the first part of the path to save ("/opt/docker/notebooks/data/") is always the same

To refind the image, we can connect to server, and the PNG file is in /home/factory02/notebook

In [ ]:
// Path
val ndviPath_server = "/opt/docker/notebooks/data/" + landsatName + ".png"

ndviPath_server: String = /opt/docker/notebooks/data/LC08_L1TP_125052_20171231_20180103_01_T1.png


In [ ]:
// Write to server
ndvi_png.write(ndviPath_server)

# In notebook, we can print the PNG..

In [ ]:
import java.io.File

import java.io.File


In [ ]:
val image_ndvi = img() // default type and size
image_ndvi.file(new File(ndviPath_server))

image_ndvi: notebook.front.SingleConnectedWidget[java.awt.image.BufferedImage]{implicit val codec: notebook.Codec[play.api.libs.json.JsValue,java.awt.image.BufferedImage]; lazy val toHtml: scala.xml.Elem; def url(u: java.net.URL): Unit; def file(f: java.io.File): Unit} = <$anon$1 widget>


In [ ]:
image_ndvi

res27: notebook.front.SingleConnectedWidget[java.awt.image.BufferedImage]{implicit val codec: notebook.Codec[play.api.libs.json.JsValue,java.awt.image.BufferedImage]; lazy val toHtml: scala.xml.Elem; def url(u: java.net.URL): Unit; def file(f: java.io.File): Unit} = <$anon$1 widget>


# To create some widgets in Hupi-Front

In Hupi-Front, we can have a widget that shows level of NDVI for each image. So for each image, we will classify NDVI into 9 classes (like what we did with the color Map) (we filter out the NaN values) :

- <= 0 
- <= 0.1 
- <= 0.2
- <= 0.3
- <= 0.4
- <= 0.5
- <= 0.6
- <= 0.7
- <= 1

In [ ]:
// Here, we convert Tile of NDVI to ArrayDouble then flatten it by using flatMap, then filter out NaN values
// We should recall this parammeter in this block, if not it will throw error Task not serializable 
val name_landsat = landsatName
val ndvi_grouped = ndvi.map(l => l._2.toArrayDouble()).flatMap(l => l)
.filter(l => !l.isNaN)
// classify ndvi by its value
.map {
  case (value_ndvi) => {
    if (value_ndvi <= 0.0) {
      ("0 - NDVI <= 0.0")
    } else if (value_ndvi <= 0.1) {
      ("1 - NDVI in (0.0; 0.1]")
    } else if (value_ndvi <= 0.2) {
      ("2 - NDVI in (0.1; 0.2]")
    } else if (value_ndvi <= 0.3) {
      ("3 - NDVI in (0.2; 0.3]")
    } else if (value_ndvi <= 0.4) {
      ("4 - NDVI in (0.3; 0.4]")
    } else if (value_ndvi <= 0.5) {
      ("5 - NDVI in (0.4; 0.5]")
    } else if (value_ndvi <= 0.6) {
      ("6 - NDVI in (0.5; 0.6]")
    } else if (value_ndvi <= 0.7) {
      ("7 - NDVI in (0.6; 0.7]")
    } else {
      ("8 - NDVI > 0.7")
    }
  }
}
// reduceByKey to compute number of pixel by group
.map(l => (l, 1)).reduceByKey(_ + _).map(l => (name_landsat, l._1, l._2))
// convert to DataFrame to save in Mongo
.toDF("landsatName", "groupNDVI", "cnt")

name_landsat: String = LC08_L1TP_125052_20171231_20180103_01_T1
ndvi_grouped: org.apache.spark.sql.DataFrame = [landsatName: string, groupNDVI: string ... 1 more field]


In [ ]:
/*
// We save the collection to Mongo by appending it (if it's new image) oy by overwriting it (if old image)
ndvi_grouped.write.format("com.mongodb.spark.sql").option("uri", s"mongodb://10.100.2.7:27017/hupi.groupNDVI")
.mode("overwrite").save()
*/